In [26]:
'''Train CIFAR10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import sys
import time
import math
from tensorboardX import SummaryWriter

# from models import *
# from utils import progress_bar
from torch.autograd import Variable

In [4]:
use_cuda = torch.cuda.is_available()
print (use_cuda)
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

True
==> Preparing data..
Files already downloaded and verified


In [6]:
'''VGG11/13/16/19 in Pytorch.'''
import torch
import torch.nn as nn
from torch.autograd import Variable


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

# net = VGG('VGG11')
# x = torch.randn(2,3,32,32)
# print(net(Variable(x)).size())

In [7]:
print('==> Building model..')
net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()

==> Building model..


In [9]:
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

In [27]:
lr=0.001
writer = SummaryWriter()


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

In [22]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        #forward
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        #backward
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


In [23]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        writer.add_graph(net, inputs)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.module if use_cuda else net,
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.t7')
        best_acc = acc

In [ ]:
for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)


Epoch: 0
0 391 Loss: 0.417 | Acc: 84.375% (108/128)
1 391 Loss: 0.484 | Acc: 82.031% (210/256)
2 391 Loss: 0.486 | Acc: 83.333% (320/384)
3 391 Loss: 0.507 | Acc: 81.641% (418/512)
4 391 Loss: 0.490 | Acc: 82.969% (531/640)
5 391 Loss: 0.462 | Acc: 84.375% (648/768)
6 391 Loss: 0.459 | Acc: 84.710% (759/896)
7 391 Loss: 0.444 | Acc: 84.766% (868/1024)
8 391 Loss: 0.455 | Acc: 84.375% (972/1152)
9 391 Loss: 0.455 | Acc: 84.453% (1081/1280)
10 391 Loss: 0.445 | Acc: 84.730% (1193/1408)
11 391 Loss: 0.444 | Acc: 84.766% (1302/1536)
12 391 Loss: 0.433 | Acc: 84.976% (1414/1664)
13 391 Loss: 0.437 | Acc: 84.933% (1522/1792)
14 391 Loss: 0.435 | Acc: 85.052% (1633/1920)
15 391 Loss: 0.429 | Acc: 85.059% (1742/2048)
16 391 Loss: 0.426 | Acc: 85.202% (1854/2176)
17 391 Loss: 0.432 | Acc: 84.809% (1954/2304)
18 391 Loss: 0.430 | Acc: 84.827% (2063/2432)
19 391 Loss: 0.438 | Acc: 84.922% (2174/2560)
20 391 Loss: 0.450 | Acc: 84.487% (2271/2688)
21 391 Loss: 0.449 | Acc: 84.624% (2383/2816)
22 3

174 391 Loss: 0.431 | Acc: 85.076% (19057/22400)
175 391 Loss: 0.431 | Acc: 85.054% (19161/22528)
176 391 Loss: 0.431 | Acc: 85.033% (19265/22656)
177 391 Loss: 0.431 | Acc: 85.038% (19375/22784)
178 391 Loss: 0.431 | Acc: 85.052% (19487/22912)
179 391 Loss: 0.430 | Acc: 85.065% (19599/23040)
180 391 Loss: 0.430 | Acc: 85.070% (19709/23168)
181 391 Loss: 0.430 | Acc: 85.079% (19820/23296)
182 391 Loss: 0.431 | Acc: 85.084% (19930/23424)
183 391 Loss: 0.430 | Acc: 85.071% (20036/23552)
184 391 Loss: 0.430 | Acc: 85.072% (20145/23680)
185 391 Loss: 0.431 | Acc: 85.043% (20247/23808)
186 391 Loss: 0.431 | Acc: 85.069% (20362/23936)
187 391 Loss: 0.431 | Acc: 85.065% (20470/24064)
188 391 Loss: 0.431 | Acc: 85.049% (20575/24192)
189 391 Loss: 0.430 | Acc: 85.062% (20687/24320)
190 391 Loss: 0.430 | Acc: 85.070% (20798/24448)
191 391 Loss: 0.430 | Acc: 85.067% (20906/24576)
192 391 Loss: 0.430 | Acc: 85.067% (21015/24704)
193 391 Loss: 0.430 | Acc: 85.068% (21124/24832)
194 391 Loss: 0.431 

342 391 Loss: 0.431 | Acc: 85.074% (37351/43904)
343 391 Loss: 0.431 | Acc: 85.074% (37460/44032)
344 391 Loss: 0.431 | Acc: 85.068% (37566/44160)
345 391 Loss: 0.431 | Acc: 85.068% (37675/44288)
346 391 Loss: 0.432 | Acc: 85.066% (37783/44416)
347 391 Loss: 0.432 | Acc: 85.064% (37891/44544)
348 391 Loss: 0.432 | Acc: 85.040% (37989/44672)
349 391 Loss: 0.433 | Acc: 85.020% (38089/44800)
350 391 Loss: 0.432 | Acc: 85.027% (38201/44928)
351 391 Loss: 0.432 | Acc: 85.032% (38312/45056)
352 391 Loss: 0.432 | Acc: 85.046% (38427/45184)
353 391 Loss: 0.432 | Acc: 85.026% (38527/45312)
354 391 Loss: 0.432 | Acc: 85.018% (38632/45440)
355 391 Loss: 0.432 | Acc: 85.020% (38742/45568)
356 391 Loss: 0.432 | Acc: 85.016% (38849/45696)
357 391 Loss: 0.433 | Acc: 85.006% (38953/45824)
358 391 Loss: 0.433 | Acc: 84.993% (39056/45952)
359 391 Loss: 0.433 | Acc: 84.985% (39161/46080)
360 391 Loss: 0.433 | Acc: 84.979% (39267/46208)
361 391 Loss: 0.433 | Acc: 84.984% (39378/46336)
362 391 Loss: 0.433 

27 391 Loss: 0.390 | Acc: 86.607% (3104/3584)
28 391 Loss: 0.392 | Acc: 86.584% (3214/3712)
29 391 Loss: 0.393 | Acc: 86.562% (3324/3840)
30 391 Loss: 0.395 | Acc: 86.442% (3430/3968)
31 391 Loss: 0.393 | Acc: 86.450% (3541/4096)
32 391 Loss: 0.396 | Acc: 86.364% (3648/4224)
33 391 Loss: 0.394 | Acc: 86.443% (3762/4352)
34 391 Loss: 0.395 | Acc: 86.362% (3869/4480)
35 391 Loss: 0.396 | Acc: 86.372% (3980/4608)
36 391 Loss: 0.396 | Acc: 86.339% (4089/4736)
37 391 Loss: 0.394 | Acc: 86.451% (4205/4864)
38 391 Loss: 0.394 | Acc: 86.498% (4318/4992)
39 391 Loss: 0.393 | Acc: 86.523% (4430/5120)
40 391 Loss: 0.393 | Acc: 86.490% (4539/5248)
41 391 Loss: 0.390 | Acc: 86.644% (4658/5376)
42 391 Loss: 0.390 | Acc: 86.664% (4770/5504)
43 391 Loss: 0.392 | Acc: 86.559% (4875/5632)
44 391 Loss: 0.392 | Acc: 86.562% (4986/5760)
45 391 Loss: 0.390 | Acc: 86.651% (5102/5888)
46 391 Loss: 0.390 | Acc: 86.735% (5218/6016)
47 391 Loss: 0.393 | Acc: 86.621% (5322/6144)
48 391 Loss: 0.392 | Acc: 86.575% 

201 391 Loss: 0.402 | Acc: 86.320% (22319/25856)
202 391 Loss: 0.402 | Acc: 86.322% (22430/25984)
203 391 Loss: 0.402 | Acc: 86.336% (22544/26112)
204 391 Loss: 0.402 | Acc: 86.322% (22651/26240)
205 391 Loss: 0.402 | Acc: 86.317% (22760/26368)
206 391 Loss: 0.402 | Acc: 86.334% (22875/26496)
207 391 Loss: 0.402 | Acc: 86.336% (22986/26624)
208 391 Loss: 0.402 | Acc: 86.334% (23096/26752)
209 391 Loss: 0.401 | Acc: 86.328% (23205/26880)
210 391 Loss: 0.401 | Acc: 86.334% (23317/27008)
211 391 Loss: 0.401 | Acc: 86.332% (23427/27136)
212 391 Loss: 0.401 | Acc: 86.319% (23534/27264)
213 391 Loss: 0.401 | Acc: 86.303% (23640/27392)
214 391 Loss: 0.401 | Acc: 86.312% (23753/27520)
215 391 Loss: 0.401 | Acc: 86.321% (23866/27648)
216 391 Loss: 0.401 | Acc: 86.319% (23976/27776)
217 391 Loss: 0.402 | Acc: 86.292% (24079/27904)
218 391 Loss: 0.402 | Acc: 86.269% (24183/28032)
219 391 Loss: 0.402 | Acc: 86.278% (24296/28160)
220 391 Loss: 0.402 | Acc: 86.277% (24406/28288)
221 391 Loss: 0.401 

...

27 391 Loss: 0.203 | Acc: 92.690% (3322/3584)
28 391 Loss: 0.202 | Acc: 92.780% (3444/3712)
29 391 Loss: 0.201 | Acc: 92.812% (3564/3840)
30 391 Loss: 0.202 | Acc: 92.767% (3681/3968)
31 391 Loss: 0.205 | Acc: 92.725% (3798/4096)
32 391 Loss: 0.204 | Acc: 92.756% (3918/4224)
33 391 Loss: 0.202 | Acc: 92.900% (4043/4352)
34 391 Loss: 0.200 | Acc: 92.991% (4166/4480)
35 391 Loss: 0.198 | Acc: 92.969% (4284/4608)
36 391 Loss: 0.199 | Acc: 92.948% (4402/4736)
37 391 Loss: 0.199 | Acc: 92.969% (4522/4864)
38 391 Loss: 0.200 | Acc: 92.989% (4642/4992)
39 391 Loss: 0.199 | Acc: 92.969% (4760/5120)
40 391 Loss: 0.200 | Acc: 92.931% (4877/5248)
41 391 Loss: 0.200 | Acc: 92.913% (4995/5376)
42 391 Loss: 0.200 | Acc: 92.951% (5116/5504)
43 391 Loss: 0.200 | Acc: 92.915% (5233/5632)
44 391 Loss: 0.198 | Acc: 92.969% (5355/5760)
45 391 Loss: 0.198 | Acc: 92.952% (5473/5888)
46 391 Loss: 0.196 | Acc: 93.052% (5598/6016)
47 391 Loss: 0.196 | Acc: 93.083% (5719/6144)
48 391 Loss: 0.195 | Acc: 93.048% 

199 391 Loss: 0.204 | Acc: 92.754% (23745/25600)
200 391 Loss: 0.203 | Acc: 92.767% (23867/25728)
201 391 Loss: 0.203 | Acc: 92.764% (23985/25856)
202 391 Loss: 0.203 | Acc: 92.761% (24103/25984)
203 391 Loss: 0.203 | Acc: 92.762% (24222/26112)
204 391 Loss: 0.203 | Acc: 92.771% (24343/26240)
205 391 Loss: 0.203 | Acc: 92.779% (24464/26368)
206 391 Loss: 0.203 | Acc: 92.784% (24584/26496)
207 391 Loss: 0.203 | Acc: 92.792% (24705/26624)
208 391 Loss: 0.202 | Acc: 92.808% (24828/26752)
209 391 Loss: 0.202 | Acc: 92.790% (24942/26880)
210 391 Loss: 0.203 | Acc: 92.802% (25064/27008)
211 391 Loss: 0.203 | Acc: 92.792% (25180/27136)
212 391 Loss: 0.203 | Acc: 92.789% (25298/27264)
213 391 Loss: 0.203 | Acc: 92.797% (25419/27392)
214 391 Loss: 0.203 | Acc: 92.798% (25538/27520)
215 391 Loss: 0.204 | Acc: 92.781% (25652/27648)
216 391 Loss: 0.204 | Acc: 92.785% (25772/27776)
217 391 Loss: 0.203 | Acc: 92.804% (25896/27904)
218 391 Loss: 0.203 | Acc: 92.794% (26012/28032)
219 391 Loss: 0.203 

368 391 Loss: 0.211 | Acc: 92.552% (43714/47232)
369 391 Loss: 0.211 | Acc: 92.557% (43835/47360)
370 391 Loss: 0.211 | Acc: 92.554% (43952/47488)
371 391 Loss: 0.211 | Acc: 92.547% (44067/47616)
372 391 Loss: 0.211 | Acc: 92.554% (44189/47744)
373 391 Loss: 0.212 | Acc: 92.541% (44301/47872)
374 391 Loss: 0.212 | Acc: 92.544% (44421/48000)
375 391 Loss: 0.212 | Acc: 92.543% (44539/48128)
376 391 Loss: 0.212 | Acc: 92.542% (44657/48256)
377 391 Loss: 0.212 | Acc: 92.543% (44776/48384)
378 391 Loss: 0.212 | Acc: 92.536% (44891/48512)
379 391 Loss: 0.212 | Acc: 92.527% (45005/48640)
380 391 Loss: 0.212 | Acc: 92.520% (45120/48768)
381 391 Loss: 0.212 | Acc: 92.523% (45240/48896)
382 391 Loss: 0.212 | Acc: 92.528% (45361/49024)
383 391 Loss: 0.212 | Acc: 92.531% (45481/49152)
384 391 Loss: 0.212 | Acc: 92.518% (45593/49280)
385 391 Loss: 0.212 | Acc: 92.517% (45711/49408)
386 391 Loss: 0.212 | Acc: 92.523% (45832/49536)
387 391 Loss: 0.212 | Acc: 92.522% (45950/49664)
388 391 Loss: 0.212 

56 391 Loss: 0.206 | Acc: 92.859% (6775/7296)
57 391 Loss: 0.206 | Acc: 92.874% (6895/7424)
58 391 Loss: 0.206 | Acc: 92.916% (7017/7552)
59 391 Loss: 0.205 | Acc: 92.943% (7138/7680)
60 391 Loss: 0.203 | Acc: 92.982% (7260/7808)
61 391 Loss: 0.203 | Acc: 92.994% (7380/7936)
62 391 Loss: 0.205 | Acc: 92.907% (7492/8064)
63 391 Loss: 0.205 | Acc: 92.920% (7612/8192)
64 391 Loss: 0.205 | Acc: 92.921% (7731/8320)
65 391 Loss: 0.206 | Acc: 92.850% (7844/8448)
66 391 Loss: 0.208 | Acc: 92.794% (7958/8576)
67 391 Loss: 0.209 | Acc: 92.785% (8076/8704)
68 391 Loss: 0.211 | Acc: 92.686% (8186/8832)
69 391 Loss: 0.211 | Acc: 92.679% (8304/8960)
70 391 Loss: 0.210 | Acc: 92.683% (8423/9088)
71 391 Loss: 0.211 | Acc: 92.643% (8538/9216)
72 391 Loss: 0.212 | Acc: 92.616% (8654/9344)
73 391 Loss: 0.212 | Acc: 92.620% (8773/9472)
74 391 Loss: 0.211 | Acc: 92.635% (8893/9600)
75 391 Loss: 0.210 | Acc: 92.660% (9014/9728)
76 391 Loss: 0.209 | Acc: 92.715% (9138/9856)
77 391 Loss: 0.210 | Acc: 92.688% 

226 391 Loss: 0.200 | Acc: 92.993% (27020/29056)
227 391 Loss: 0.200 | Acc: 92.986% (27137/29184)
228 391 Loss: 0.200 | Acc: 92.989% (27257/29312)
229 391 Loss: 0.200 | Acc: 92.979% (27373/29440)
230 391 Loss: 0.200 | Acc: 92.982% (27493/29568)
231 391 Loss: 0.200 | Acc: 92.982% (27612/29696)
232 391 Loss: 0.200 | Acc: 92.982% (27731/29824)
233 391 Loss: 0.200 | Acc: 92.975% (27848/29952)
234 391 Loss: 0.200 | Acc: 92.962% (27963/30080)
235 391 Loss: 0.200 | Acc: 92.965% (28083/30208)
236 391 Loss: 0.200 | Acc: 92.965% (28202/30336)
237 391 Loss: 0.200 | Acc: 92.962% (28320/30464)
238 391 Loss: 0.201 | Acc: 92.949% (28435/30592)
239 391 Loss: 0.201 | Acc: 92.946% (28553/30720)
240 391 Loss: 0.201 | Acc: 92.953% (28674/30848)
241 391 Loss: 0.200 | Acc: 92.966% (28797/30976)
242 391 Loss: 0.200 | Acc: 92.969% (28917/31104)
243 391 Loss: 0.201 | Acc: 92.959% (29033/31232)
244 391 Loss: 0.201 | Acc: 92.943% (29147/31360)
245 391 Loss: 0.201 | Acc: 92.943% (29266/31488)
246 391 Loss: 0.201 